In [1]:
import sys
import os 
sys.path.insert(1, os.path.split(os.getcwd())[0])

In [2]:
from datasets.dataloader import DataLoader
from model import Trainer
from utils.utils import create_folders
from batch_gen import BatchGenerator

import torch 

from transformer import TransformerClassifier, TransfromerTrainer

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## DataLoader : 

In [14]:
class Args(): 
    def __init__(self, *args, **kwargs):
        self.train_data = 'bslcp'
        self.test_data = 'bslcp'
        self.i3d_training = 'i3d_kinetics_bslcp_981'
        self.num_in_frames = 16
        self.features_dim = 1024
        self.weights = 'opt'
        self.regression = 0 
        self.feature_normalization = 0
        self.eval_use_CP = 0

        self.action = 'train'
        self.seed = 0 
        self.refresh = 'store_true'

        ## Transformer : 
        self.nhead = 4
        self.nhid = 256
        self.dim_feedforward = 1024
        self.num_layers = 6
        self.dropout = 0

        ## MSTCN : 
        self.num_stages = 4
        self.num_layers = 10 
        self.num_f_maps = 64
        self.features_dim = 1024
        self.bz = 8 
        self.lr = 0.0005
        self.lr_mul = 1
        self.num_epochs = 50
        self.extract_epoch = 10 
        self.classification_threshold = 0.5
        
        ## Optimization
        self.n_warmup_steps = 100
        ## save model : 
        self.use_pseudo_labels = 'store_true'
        self.pretrained = False
        self.folder = ''
        
args = Args()

In [15]:
# load train dataset and test dataset

print(f'Load train data: {args.train_data}')
train_loader = DataLoader(args, args.train_data, 'train')
print(f'Load test data: {args.test_data}')
test_loader = DataLoader(args, args.test_data, 'test')

Load train data: bslcp
Load test data: bslcp


In [16]:
## Some infos : 
print("number of class : ", train_loader.num_classes)
print("cross entropy loss weigths : ", train_loader.weights)
print('number of videos in train : ', len(train_loader.vid_list))
print('number of videos in test : ', len(test_loader.vid_list))

number of class :  2
cross entropy loss weigths :  [0.11247607877029446, 0.8875239212297056]
number of videos in train :  5413
number of videos in test :  702


In [17]:
model_load_dir, model_save_dir, results_save_dir = create_folders(args, model_type = 'transformer')
print(model_save_dir)

Saved options to ./exps//models/classification/traindata_bslcp/i3d_kinetics_bslcp_981/supervised/(4,)_(10,)_(256,)_8_0.0005_weighted_opt/seed_0\opt.txt
./exps//models/classification/traindata_bslcp/i3d_kinetics_bslcp_981/supervised/(4,)_(10,)_(256,)_8_0.0005_weighted_opt/seed_0


# Transformer model : 

In [18]:
nhid = 1024  # the dimension of the feedforward network model in nn.TransformerEncoder
dim_feedforward = 1024
nlayers = 4  # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 8  # the number of heads in the multiheadattention models
dropout = 0.1  # the dropout value

nclasses = 2 # for classification task only

model = TransformerClassifier(nhead, nhid, dim_feedforward, nlayers, nclasses, dropout).to(device)

## Trainer : 

In [19]:
trainer = TransfromerTrainer(
    args.nhead,
    args.nhid,
    args.dim_feedforward,
    args.num_layers,
    nclasses,
    args.dropout, 
    device,
    train_loader.weights,
    model_save_dir
    )

Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\site-packages\tensorboard\summary\writer\event_file_writer.py", line 238, in run
    self._record_writer.write(data)
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\site-packages\tensorboard\summary\writer\record_writer.py", line 40, in write
    self._writer.write(header + header_crc + data + footer_crc)
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\site-packages\tensorboard\compat\tensorflow_stub\io\gfile.py", line 531, in write
    self.fs.write(self.filename, file_content, self.binary_mode)
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\site-packages\tensorboard\compat\tensorflow_stub\io\gfile.py", line 148, in write
    self._write(filename, file_content, "wb" if binary_mode else "w")
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\site-packag

In [20]:
eval_args = [
    args,
    model_save_dir,
    results_save_dir,
    test_loader.features_dict,
    test_loader.gt_dict,
    test_loader.eval_gt_dict,
    test_loader.vid_list,
    args.num_epochs,
    device,
    'eval',
    args.classification_threshold,
]

In [21]:
batch_gen = BatchGenerator(
        train_loader.num_classes,
        train_loader.gt_dict,
        train_loader.features_dict,
        train_loader.eval_gt_dict
        )

batch_gen.read_data(train_loader.vid_list)

In [22]:
trainer.train(
    model_save_dir,
    batch_gen,
    args.num_epochs,
    args.bz,
    args.lr,
    device,
    eval_args,
    args.lr_mul,
    args.n_warmup_steps,
    pretrained=model_load_dir)

(50/676.625) Batch: 1.1s | Total: 0:00:53 | ETA: 0:10:20 | Loss: 0.6881963610649109


KeyboardInterrupt: 